In [1]:
from riotwatcher import LolWatcher, ApiError
from riot_api import key

region = 'euw1'

lol = LolWatcher(key)

In [2]:
me = lol.summoner.by_name(region,'Katiounette')
puuid = me['puuid']

In [3]:
matchs = lol.match.matchlist_by_puuid(region,puuid, count = 100)
i = 100
while (lol.match.matchlist_by_puuid(region,puuid, count = 100, start = i) != []):
    matchs.extend(lol.match.matchlist_by_puuid(region,puuid, count = 100, start = i))
    i = i + 100
print(matchs)

['EUW1_6292868956', 'EUW1_6292795025', 'EUW1_6290869460', 'EUW1_6290774402', 'EUW1_6290767974', 'EUW1_6289709267', 'EUW1_6289659817', 'EUW1_6289581980', 'EUW1_6289288556', 'EUW1_6289270229', 'EUW1_6289215543', 'EUW1_6288423051', 'EUW1_6288364927', 'EUW1_6288238631', 'EUW1_6288186327', 'EUW1_6287531545', 'EUW1_6287059877', 'EUW1_6287006939', 'EUW1_6286714776', 'EUW1_6286665046', 'EUW1_6286606744', 'EUW1_6286560980', 'EUW1_6286529188', 'EUW1_6282616015', 'EUW1_6282527241', 'EUW1_6281409609', 'EUW1_6280987556', 'EUW1_6280908258', 'EUW1_6279756923', 'EUW1_6279713945', 'EUW1_6270239998', 'EUW1_6270170516', 'EUW1_6270106964', 'EUW1_6269848437', 'EUW1_6269794350', 'EUW1_6269744729', 'EUW1_6269707121', 'EUW1_6269497182', 'EUW1_6269472301', 'EUW1_6268941196', 'EUW1_6268872649', 'EUW1_6267679935', 'EUW1_6267613190', 'EUW1_6266419179', 'EUW1_6266327893', 'EUW1_6266313508', 'EUW1_6266221222', 'EUW1_6266151310', 'EUW1_6265551786', 'EUW1_6265508203', 'EUW1_6263879637', 'EUW1_6263746404', 'EUW1_62631

In [4]:
puuids = [puuid]

In [5]:
for i in matchs:
    p = lol.match.by_id(region,i)['metadata']['participants']
    puuids.extend(p)

In [8]:
puuids = list(set(puuids))
print(len(puuids))
print(puuids)

3342
['F4TpbVNYCzInXBD1uCqrWYkCeKN52xv85rTspUeZs4PsI-HqNbhnpI956OKGCf0bXKYxWdamvUcsPg', 'PJlq_uusZo3S3cREIV6-bCjpScOoppXTipjOUkQHRU545xMYEnJoD0ufyaVa3Xak6Q3tXDGke0RXgQ', '-LqLh_wCuNEJkx_BjNjtgO-0-n54fX5SvAyzMs7e_xWanrnswhfYCrMAkhV4lcMD67YrmVcu9Lf0Tg', 'THrZm1qRyK9hMjL7tGnFR9Nyo6Q6glh1h1zIgjV--IWHoxSoGc3LEXEhbXyuFCDjxaNd1efkB1Wi2Q', 'oJ_K5nLZIc9lYf6Ubr1S6l7FN5nIv54ilJU6OTKVhTHoUGgpkACuNqF01ZbMBQOdIwAX3PdLFy1UNw', 'AwL0YESvR2TrDwCCHG1j9HEeKedOTPonFerBtAJqFMWEOYneuOPqw0ldUoHEMvyA7dKcqmot20nz7g', 'DngIGlQbUSj2IWwAVT4dPi4VAGXLrb1VG5GqYapmHgCfagd01CCPKChzPEThnsEc3S6qZTq6_RRtvw', 'lqOLRmtUchg4wp4mFOpeCQ3q7u5SKCzRwuGkx-Smhdw1MAyz8sWl1TMl4mhEUm10DziAlpvF9roqQg', 'gaP0fjtmV_gQ5fpR9sEBOpecTCBDYTcZOlRITmbcod2N1BuLBMxc00p16oudYkAcyWcINgRX_adVAA', 'wujXEkP-FTB8yILnAnNnrKpLKR1Vl1XSvMkcH7Jilht-_o5kSnXxRaALsSh2wyPD7Ut7JtpyK5c7vQ', 'CDoWhyb6JENLZSHIUVBbRVCxih2WLXVIWyq9sh3lc61jCr-X-nMZHRQuY8tiIK7rMUuHnuzrl93kSg', 's1b-rsSdFkOq-keWJ98RCGm4hKH8RnoTNUusxpGeJ_BQRxLX7fSfnuqh_D-6zu3N_0EBCt-LpI9fQg', 'WzkBm6j1Z